In [ ]:
# Import libs
import sys
import matplotlib.pyplot as plt
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer

try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

# Declare the parameters

# name, benefit, max demand, min demand
products = [("A", 50, 100, 0),
            ("B", 40, 120, 80),
            ("C", 30, 999999999999999, 0)]

# resources are a list of simple tuples (name, capacity)
resources = [("Equipo1", 160),
             ("Equipo2", 180),
             ("Equipo3", 110)]

consumptions = {("A", "Equipo1"): 0.8,
                ("B", "Equipo1"): 0.8,
                ("C", "Equipo1"): 0.3,
                ("A", "Equipo2"): 0.6,
                ("B", "Equipo2"): 1.2,
                ("C", "Equipo2"): 0,
                ("A", "Equipo3"): 0.6,
                ("B", "Equipo3"): 1,
                ("C", "Equipo3"): 0.6}

# Create the model with constraints and objective
def create_model():
    mdl = Model(name="guia5problematipo2")

    produccion_vars = mdl.continuous_var_dict(products, name='produccion')

    # --- constraints ---

    # --- resources disp equipo ---
    mdl.add_constraints((mdl.sum(produccion_vars[p] * consumptions[p[0], res[0]] for p in products) <= res[1], 'Disp_%s' % res[0]) for res in resources)

    # max demand
    mdl.add_constraints((produccion_vars[p] <= p[2], 'DemandMax_%s' % p[0]) for p in products)

    # min demand
    mdl.add_constraints((produccion_vars[p] >= p[3], 'DemandMin_%s' % p[0]) for p in products)

    # --- print information ---
    mdl.print_information()

    total_benefit = mdl.sum(produccion_vars[p] * p[1] for p in products)

    # --- set the objective ---
    mdl.maximize(total_benefit)

    return mdl, produccion_vars, products

# Solve the model
def solve_model(mdl):
    solution = mdl.solve()

    if not solution:
        print("Model cannot be solved.")
        sys.exit(1)

    obj = mdl.objective_value

    print("* Production model solved with objective: {:g}".format(obj))
    print("* Total benefit=%g" % mdl.objective_value)
    for p in products:
        print("Production of {product}: {prod_var}".format(product=p[0], prod_var=produccion_vars[p].solution_value))